## 8.1.3. Важность стратегии выбора

Для управления величиной случайности в процессе выбора введем параметр,
который назовем температурой softmax, характеризующий энтропию распределения вероятностей, используемую для выбора: она будет определять степень необычности или предсказуемости выбора следующего символа. С учетом значения
temperature и на основе оригинального распределения вероятностей (результата
функции softmax модели) будет вычисляться новое распределение путем взвешивания вероятностей, как показано ниже.

In [5]:
# Взвешивание распределения вероятностей с учетом значения температуры

import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):  # original_distribution — это одномерный массив Numpy
                                                                      # значений вероятностей, сумма которых должна быть равна 1

    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)                      # Возвращает новую, взвешенную версию оригинального распределения. Сумма вероятностей
                                                                      # в новом распределении может получиться больше 1, поэтому разделим элементы вектора на сумму,
                                                                      # чтобы получить новое распределение


## 8.1.4. Реализация посимвольной генерации текста на основе LSTM

Воплотим эти идеи на практике в реализации с Keras. Первое, что нам понадобится, — это много текстовых данных, на которых можно было бы обучить языковую
модель. Для этого можно использовать любой большой текстовый файл или набор текстовых файлов, например статьи из Википедии, роман «Властелин колец»
и т. д. В данном примере мы используем тексты из произведений Ницше, немецкого
философа конца XIX века (в переводе на английский язык). Таким образом, в результате обучения у нас получится языковая модель, обладающая специфическими
особенностями, характерными для произведений Ницше, а не обобщенная модель
английского языка.


In [6]:
# Загрузка и парсинг исходного текстового файла

import tensorflow.keras.utils
import numpy as np
path = tensorflow.keras.utils.get_file(
        'nietzsche.txt',
        origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))  

Corpus length: 600893


In [7]:
# Векторизация последовательностей символов

maxlen = 60                                                                     # Извлечение последовательностей по 60 символов

step = 3                                                                        # Новые последовательности выбираются через каждые 3 символа

sentences = []                                                                  # Хранение извлеченных последовательностей

next_chars = []                                                                 # Хранение целей (символов, следующих за последовательностями)

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))                                                 # Список уникальных символов в корпусе
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)                # Словарь, отображающий уникальные символы в их
                                                                                  # индексы в списке «chars»

print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1                                       # Прямое кодирование символов
                                                                                  # в бинарные массивы



Number of sequences: 200278
Unique characters: 57
Vectorization...


### Конструирование сети

Эта сеть состоит из единственного слоя LSTM, за которым следует классификатор
Dense с функцией softmax выбора из всех возможных символов. Но имейте в виду,
что рекуррентные нейронные сети не единственный способ генерирования последовательностей данных; одномерные сверточные сети тоже показали превосходные
результаты в решении этой задачи.

In [8]:
# Модель с единственным слоем LSTM для предсказания следующего символа

from keras import layers

model = tensorflow.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Так как цели имеют формат прямого кодирования, используем для обучения модели функцию потерь categorical_crossentropy.

In [9]:
# Конфигурация компилируемой модели

optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### Обучение модели и извлечение образцов из нее

Имея обученную модель и фрагмент начального текста, можно сгенерировать новый текст, выполнив следующие пункты:

1. Извлечь из модели распределение вероятностей следующего символа для имеющегося на данный момент сгенерированного текста.

2. Выполнить взвешивание распределения с заданной температурой.

3. Выбрать следующий символ в соответствии с вновь взвешенным распределением вероятностей.

4. Добавить новый символ в конец текста.

Вот код, который мы используем для взвешивания оригинального распределения
вероятностей, возвращаемого моделью, и извлечения индекса символа (функция
выборки).

In [10]:
# Функция выборки следующего символа с учетом прогнозов модели

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Наконец, следующий цикл повторяет обучение и генерирует текст. Для начала
сгенерируем текст, использовав разные температуры после каждой эпохи. Это позволит вам увидеть, как меняется генерируемый тест по мере схождения модели
и как температура влияет на стратегию выбора.

In [13]:
# Цикл генерации текста

import random
import sys
temperatures = [0.2, 0.5, 1.0, 1.2]
for epoch in range(1, 3):                                                      # Обучение модели в течение 60 эпох
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)                                   # Выполнение одной итерации обучения

start_index = random.randint(0, len(text) - maxlen - 1)                         # Выбор случайного начального текста
generated_text = text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')




epoch 1
1565/1565 [==============================] - 25s 16ms/step - loss: 1.4321
epoch 2
1565/1565 [==============================] - 24s 16ms/step - loss: 1.4151
--- Generating with seed: "dividual man himself now goes through too many
stages of inn"


In [15]:
import random
import sys
temperatures = [0.2, 0.5, 1.0, 1.2]
for temperature in temperatures:
    print('------ temperature:', temperature)   
    sys.stdout.write(generated_text)    
    for i in range(400):                                                        # Генерация 400 символов, начиная с начального текста
        sampled = np.zeros((1, maxlen, len(chars)))                             # Прямое кодирование символов, сгенерированных до сих пор
        for t, char in enumerate(generated_text):                               
            sampled[0, t, char_indices[char]] = 1.    
        preds = model.predict(sampled, verbose=0)[0]                            # Выбор следующего символа
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)

------ temperature: 0.2
 of the contrary of the soul, the thinker of the subject the strong the strong the stands and the strong the promaned the same things the sentiments of the more and the does a things and the fact the stands the precisely the strong the sentiments of the fact the consequently the strong the contrary of the stand the sentiments of the strong the subject the stand in the fact the fact the strong of the consequently in the most consequently the strong the most------ temperature: 0.5
he consequently in the most consequently the strong the most sure the principle and will in the fact the objection of the interpretely the play of the general superficial the man from the philosophers or the suddenly the decempt of the senses. the same state the soul and man and stiftion the contrary of the deed and the subject to knowledge, the interes about the superfulness of the privine of the one which the evils and will the the man while the fact the------ temperature: 1.0
 one whi